<a href="https://colab.research.google.com/github/SriNihitha12/BigdataProject/blob/main/Traffic_Prediction_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>